<a href="https://colab.research.google.com/github/nxvya/MAINPROJECT/blob/main/CNN%2BTCN%20UPDATED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files

# Use the 'files.upload()' method to upload files
uploaded = files.upload()

# Access the uploaded file
for filename, content in uploaded.items():
    with open(filename, 'wb') as f:
        f.write(content)

Saving phising.csv to phising.csv


In [3]:
pip install tensorflow numpy pandas


In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense


In [5]:
import pandas as pd
# Replace 'your_dataset.csv' with the
df = pd.read_csv('phising.csv')

In [6]:
# Import the LabelEncoder module
!pip install scikit-learn
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder object
label_encoder = LabelEncoder()

# Apply the fit_transform method to the 'type' column of the df DataFrame
df['type'] = label_encoder.fit_transform(df['type'])
label_encoder = LabelEncoder()
df['type'] = label_encoder.fit_transform(df['type'])


In [7]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [8]:
max_words = 10000  # Set the maximum number of words in your vocabulary
max_len = 100      # Set the maximum length of the URL sequence


In [9]:
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(train_df['url'])

X_train = tokenizer.texts_to_sequences(train_df['url'])
X_test = tokenizer.texts_to_sequences(test_df['url'])

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [15]:
embedding_dim = 50  # Set the embedding dimension
filters = 64        # Set the number of filters in the Conv1D layer
kernel_size = 3     # Set the kernel size in the Conv1D layer
tcn_filters = 64    # Set the number of filters in the TCN layer
num_classes = 4     # Number of classes in your dataset

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=tcn_filters, activation='relu'))  # TCN layer
model.add(Dense(units=num_classes, activation='softmax'))  # Output layer for multi-class classification

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])



In [16]:
y_train = train_df['type']
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.1)

Epoch 1/5
14652/14652 [==============================] - 270s 18ms/step - loss: 0.1433 - accuracy: 0.9468 - val_loss: 0.1068 - val_accuracy: 0.9599
Epoch 2/5
14652/14652 [==============================] - 275s 19ms/step - loss: 0.0968 - accuracy: 0.9631 - val_loss: 0.1010 - val_accuracy: 0.9621
Epoch 3/5
14652/14652 [==============================] - 271s 18ms/step - loss: 0.0858 - accuracy: 0.9670 - val_loss: 0.1018 - val_accuracy: 0.9630
Epoch 4/5
14652/14652 [==============================] - 270s 18ms/step - loss: 0.0795 - accuracy: 0.9694 - val_loss: 0.1039 - val_accuracy: 0.9621
Epoch 5/5
14652/14652 [==============================] - 279s 19ms/step - loss: 0.0751 - accuracy: 0.9708 - val_loss: 0.1060 - val_accuracy: 0.9621


In [17]:
y_test = test_df['type']
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

4070/4070 [==============================] - 16s 4ms/step - loss: 0.1089 - accuracy: 0.9616
Test Accuracy: 96.16%


In [18]:
new_data = ["videosurf.com/pat-patterson-1447461","whttp://style.org.hc360.com/css/detail/mysite/siteconfig/pro_control.css", "http://pastehtml.com/view/blpo63jy9.html"]
sequences = tokenizer.texts_to_sequences(new_data)
padded_sequences = pad_sequences(sequences, maxlen=max_len)
predictions = model.predict(padded_sequences)

1/1 [==============================] - 0s 109ms/step


In [19]:
predicted_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=1))

print(f'Predicted Labels: {predicted_labels}')

Predicted Labels: [0 2 3]


In [21]:
label_indices = np.argmax(predictions, axis=1)
print(f'Predicted Labels: {predicted_labels}')
print(f'Label Indices: {label_indices}')

Predicted Labels: [0 2 3]
Label Indices: [0 2 3]


In [22]:
# Create a dictionary mapping label indices to labels
index_to_label = {0: 'benign', 1: 'defacemnt', 2: 'malware', 3: 'phishing'}

# Map label indices to labels in predictions
predicted_labels_mapped = [index_to_label[index] for index in label_indices]

print(f'Predicted Labels (Mapped): {predicted_labels_mapped}')

Predicted Labels (Mapped): ['benign', 'malware', 'phishing']
